In [1]:
# !wget https://huggingface.co/datasets/TokenBender/unnatural_code_instructions_20M/resolve/main/unnatural_training_data_unique.jsonl

In [2]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai4-aiservices834896161.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

engine = 'gpt-35-turbo'

In [4]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
instructions = []
with open('unnatural_training_data_unique.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        l = l['text'].split('### Response')[0].split('### Instruction:')[1].strip().replace('\\n', '').strip()
        instructions.append(l)
        
len(instructions)

112345

In [6]:
from depth import createConstraintsPrompt, createDeepenPrompt, createConcretizingPrompt, createReasoningPrompt
from breadth import createBreadthPrompt

In [7]:
!mkdir code-instructions
# !rm output/*

mkdir: cannot create directory ‘code-instructions’: File exists


In [8]:
def generate(instruction, index):
    filename = f'code-instructions/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        evol_prompts = []
        evol_prompts.append(createConstraintsPrompt(instruction))
        evol_prompts.append(createDeepenPrompt(instruction))
        evol_prompts.append(createConcretizingPrompt(instruction))
        evol_prompts.append(createReasoningPrompt(instruction))
        evol_prompts.append(createBreadthPrompt(instruction))

        selected_evol_prompt = random.choice(evol_prompts)
        ins = predict(selected_evol_prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [9]:
generate(instructions[5], 5)

In [10]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(instructions) // 4, max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

 31%|███████████▌                         | 2910/9362 [00:00<00:00, 9707.76it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'
The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
unsupported operand type(s) for +: 'NoneType' and 'str'


 63%|██████████████████████▏            | 5932/9362 [4:01:28<9:03:19,  9.50s/it]

'content'


 73%|███████████████████████▍        | 6874/9362 [7:26:44<132:02:53, 191.07s/it]

...

 78%|███████████████████████████▏       | 7265/9362 [8:42:28<8:26:14, 14.48s/it]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 81%|████████████████████████████▎      | 7572/9362 [9:47:15<7:24:01, 14.88s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|████████████████████████████████████| 9362/9362 [16:28:15<00:00,  6.33s/it]
